# Proyecto Final, tarea SQL

## Introducción

El coronavirus tomó al mundo entero por sorpresa, cambiando la rutina diaria de todos y todas. Los habitantes de las ciudades ya no pasaban su tiempo libre fuera, yendo a cafés y centros comerciales; sino que más gente se quedaba en casa, leyendo libros. Eso atrajo la atención de las startups (empresas emergentes) que se apresuraron a desarrollar nuevas aplicaciones para los amantes de los libros.

Tenemos una base de datos de uno de los servicios que compiten en este mercado. Contiene datos sobre libros, editoriales, autores y calificaciones de clientes y reseñas de libros. Esta información se utilizará para generar una propuesta de valor para un nuevo producto.

Vamos a encontrar lo siguiente:
- Número de libros publicados después del 1 de enero de 2000.
- Número de reseñas de usuarios y la calificación promedio para cada libro.
- La editorial que ha publicado el mayor número de libros con más de 50 páginas (excluir folletos y similares).
- Autor que tiene la más alta calificación promedio del libro. Miraremos solo los libros con al menos 50 calificaciones.
- Número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

Al final de este cuaderno, incluiremos conclusiones a partir de nuestros hallazgos para una propuesta de valor.

**Descripción de los datos**

Tabla `**books**` contiene datos sobre libros:

    - `book_id`: identificación del libro
    - `author_id`: identificación del autor o autora
    - `title`: título
    - `num_pages`: número de páginas
    - `publication_date`: fecha de la publicación
    - `publisher_id`: identificación de la editorial

Tabla `**authors**` contiene datos sobre autores:

    - `author_id`: identificación del autor o autora
    - `author`: el autor o la autora

Tabla `**publishers**` contiene datos sobre editoriales:

    - `publisher_id`: identificación de la editorial
    - `publisher`: la editorial

Tabla `**ratings**` contiene datos sobre las calificaciones de usuarios:

    - `rating_id`: identificación de la calificación
    - `book_id`: identificación del libro
    - `username`: el nombre del usuario que revisó el libro
    - `rating`: calificación

Tabla `**reviews**` contiene datos sobre las reseñas de los y las clientes:

    - `review_id`: identificación de la reseña
    - `book_id`: identificación del libro
    - `username`: el nombre del usuario que revisó el libro
    - `text`: el texto de la reseña



## Descargar Paquetes y Librerías

In [15]:
import pandas as pd
from sqlalchemy import create_engine

import dotenv
dotenv.load_dotenv()

import os

os.getenv('user')

'practicum_student'

## Conectarse a la Base de Datos

In [16]:
db_config = {'user': os.getenv('user') ,                             # nombre de usuario
             'pwd': os.getenv('pwd'),                       # contraseña
             'host': os.getenv('host'),
             'port': os.getenv('port'),                                            # puerto de conexión
             'db': os.getenv('db')                    # nombre de la base de datos
            }          

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db']
                                                        )

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Estudio de las tablas

Vamos a observar cada tabla y sus primeras filas, y entender la relacion entre ellas. 

In [17]:
# imprime las primeras filas de la tabla books

query_1 = 'SELECT * FROM authors'
books_df = pd.io.sql.read_sql(query_1, con = engine)
books_df.head(5)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [5]:
# imprime las primeras filas de la tabla authors

query_2 = 'SELECT * FROM authors'
books_df = pd.io.sql.read_sql(query_2, con = engine)
books_df.head(5)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [6]:
# imprime las primeras filas de la tabla ratings

query_3 = 'SELECT * FROM ratings'
books_df = pd.io.sql.read_sql(query_3, con = engine)
books_df.head(5)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [7]:
# imprime las primeras filas de la tabla reviews

query_4 = 'SELECT * FROM reviews'
books_df = pd.io.sql.read_sql(query_4, con = engine)
books_df.head(5)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [8]:
# imprime las primeras filas de la tabla publishers

query_5 = 'SELECT * FROM publishers'
books_df = pd.io.sql.read_sql(query_5, con = engine)
books_df.head(5)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Hallazgos:
    
    Las tablas están relacionadas de la siguiente manera: 
        
        - books y authors se relacionan por la clave author_id
        - books y publishers se relacionan por la clave publisher_id
        - book, reviews, y ratings se relacionan entre sí por la clave book_id

## Consultas en SQL

Vamos a realizar cinco consultas de SQL, cada una con una tarea/objetivo puntual.

### Tarea 1

Encuentra el número de libros publicados después del 1 de enero de 2000.

In [53]:
task_1 = """

SELECT
    COUNT(*)
FROM
    books
WHERE
    CAST(publication_date AS date) > '2000-01-01'

"""

number_books = pd.io.sql.read_sql(task_1, con = engine)
print('Número de libros publicados después del 1 de enero de 2000:\n\n', number_books)

Número de libros publicados después del 1 de enero de 2000:
    count
0    819


### Tarea 2

Encuentra el número de reseñas de usuarios y la calificación promedio para cada libro.

In [80]:
task_2 = """

SELECT
    ratings.book_id AS book_id,
    AVG(ratings.rating) AS rating_avg,
    COUNT(reviews.review_id) AS review_count
FROM
    ratings
    FULL JOIN reviews 
    ON (ratings.book_id = reviews.book_id)
GROUP BY
    ratings.book_id
ORDER BY
    COUNT(reviews.review_id) DESC
"""

number_review_rating_per_book = pd.io.sql.read_sql(task_2, con = engine)
print('Número de reseñas de usuarios y la calificación promedio para cada libro:\n\n',number_review_rating_per_book.head(20))

Número de reseñas de usuarios y la calificación promedio para cada libro:

     book_id  rating_avg  review_count
0       948    3.662500          1120
1       750    4.125000           528
2       673    3.825581           516
3       302    4.414634           492
4       299    4.287500           480
5        75    3.678571           420
6       301    4.186667           375
7       779    4.080645           372
8        79    3.729730           370
9       722    4.391892           370
10      300    4.246575           365
11      405    3.901408           355
12      696    3.830508           354
13      627    3.789474           342
14      733    3.750000           336
15      545    3.787879           330
16      656    4.264151           318
17      488    3.622951           305
18      207    3.395833           288
19      963    3.977273           264


In [81]:
# visualizamos el mismo resultado, pero ordenado por las 20 mejores calificaciones

task_2_sub_1 = """

SELECT
    ratings.book_id AS book_id,
    AVG(ratings.rating) AS rating_avg,
    COUNT(reviews.review_id) AS review_count
FROM
    ratings
    FULL JOIN reviews 
    ON (ratings.book_id = reviews.book_id)
GROUP BY
    ratings.book_id
ORDER BY
    AVG(ratings.rating) DESC
"""

number_review_rating_per_book_sub_1 = pd.io.sql.read_sql(task_2_sub_1, con = engine)
print('Número de reseñas de usuarios y la calificación promedio para cada libro:\n\n',number_review_rating_per_book_sub_1.head(20))

Número de reseñas de usuarios y la calificación promedio para cada libro:

     book_id  rating_avg  review_count
0       518         5.0             4
1       732         5.0             4
2       347         5.0             6
3       610         5.0             4
4       330         5.0             2
5       229         5.0             4
6       275         5.0             4
7       553         5.0            12
8       213         5.0             4
9       993         5.0             4
10      972         5.0             4
11       57         5.0             4
12      224         5.0             4
13       17         5.0            16
14      902         5.0             4
15      459         5.0             4
16      297         5.0             4
17      182         5.0             4
18      136         5.0             4
19      418         5.0             4


### Tarea 3

Identifica la editorial que ha publicado el mayor número de libros con más de 50 páginas (esto te ayudará a excluir folletos y publicaciones similares de tu análisis)

In [87]:
task_3 = """

SELECT
    publishers.publisher AS publisher,
    COUNT(books.book_id) AS published_books
FROM
    books
    FULL JOIN publishers 
    ON (books.publisher_id = publishers.publisher_id)
WHERE
    books.num_pages > 50
GROUP BY
    publishers.publisher
ORDER BY
    published_books DESC
"""

top_publishes_fifty_pages = pd.io.sql.read_sql(task_3, con = engine)
print('Editoriales con mayor número de libros publicados,\nLibros con más de 50 páginas:\n\n',top_publishes_fifty_pages.head(20))

Editoriales con mayor número de libros publicados.
Libros con más de 50 páginas:

                     publisher  published_books
0               Penguin Books               42
1                     Vintage               31
2    Grand Central Publishing               25
3            Penguin Classics               24
4            Ballantine Books               19
5                      Bantam               19
6                     Berkley               17
7          St. Martin's Press               14
8               Berkley Books               14
9                       Delta               13
10  William Morrow Paperbacks               13
11           Harper Perennial               12
12              HarperCollins               12
13  Little  Brown and Company               12
14                  Tor Books               11
15             Back Bay Books               11
16                    Penguin               10
17                     Signet               10
18                    Ve

### Tarea 4

Identifica al autor que tiene la más alta calificación promedio del libro: mira solo los libros con al menos 50 calificaciones.

In [90]:
task_4 = """

SELECT
    authors.author AS author,
    COUNT(ratings.rating_id) AS total_ratings,
    AVG(ratings.rating) AS rating_avg
    
FROM
    authors FULL JOIN books ON authors.author_id = books.author_id
    FULL JOIN ratings ON ratings.book_id = books.book_id 
GROUP BY
    authors.author
HAVING
    COUNT(ratings.rating_id) > 50
ORDER BY
    rating_avg DESC

    
"""

top_authors_best_rating = pd.io.sql.read_sql(task_4, con = engine)
print('Autores con las más altas calificaciones promedio de libros\n\n',top_authors_best_rating.head(10))

Autores con las más altas calificaciones promedio de libros

                               author  total_ratings  rating_avg
0         J.K. Rowling/Mary GrandPré            312    4.288462
1                    Agatha Christie             53    4.283019
2  Markus Zusak/Cao Xuân Việt Khương             53    4.264151
3                     J.R.R. Tolkien            166    4.240964
4           Roald Dahl/Quentin Blake             62    4.209677
5                  Louisa May Alcott             54    4.203704
6                       Rick Riordan             84    4.130952
7                      Arthur Golden             56    4.107143
8                       Stephen King            106    4.009434
9                       John Grisham             70    3.971429


### Tarea 5

Encuentra el número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros.

In [95]:
task_5 = """

SELECT
    reviews.username AS username,
    COUNT(reviews.review_id) AS total_books_reviewed,
    COUNT(ratings.rating_id) AS total_books_rated
FROM
    reviews INNER JOIN ratings ON reviews.book_id = ratings.book_id
GROUP BY
    reviews.username
HAVING
    COUNT(ratings.rating_id) > 50

"""

avg_review_count = pd.io.sql.read_sql(task_5, con = engine)
print('Número de reseñas de texto realizadas por usuarios que calificaron más de 50 libros:\n\n',avg_review_count.head(20))

Número de reseñas de texto realizadas por usuarios que calificaron más de 50 libros:

           username  total_books_reviewed  total_books_rated
0        zjohnston                   199                199
1          jnelson                   302                302
2     lewisdesiree                   172                172
3    patrickhudson                   141                141
4     valenciaanne                   293                293
5      kristykirby                   315                315
6      conradanita                    91                 91
7     martinyvonne                   208                208
8        patrick05                   169                169
9      lunakathryn                   132                132
10     robertwells                   106                106
11   serranoangela                   115                115
12       emcdaniel                    82                 82
13      annawarner                   185                185
14         ke

In [96]:
# Encontramos el promedio de la cantidad de reseñas que hacen los usuarios

task_5_sub_1 = """

SELECT
    AVG(SUBQUERY.total_books_reviewed) AS average_of_reviews
FROM
    (SELECT
        reviews.username AS username,
        COUNT(reviews.review_id) AS total_books_reviewed,
        COUNT(ratings.rating_id) AS total_books_rated
    FROM
        reviews FULL JOIN ratings ON reviews.book_id = ratings.book_id
    GROUP BY
        reviews.username
    HAVING
        COUNT(ratings.rating_id) > 50) AS SUBQUERY

"""

avg_review_count_sub_1 = pd.io.sql.read_sql(task_5_sub_1, con = engine)
print('Promedio de número de reseñas de texto realizadas por usuarios que calificaron más de 50 libros:\n\n',avg_review_count_sub_1.head(20))

Promedio de número de reseñas de texto realizadas por usuarios que calificaron más de 50 libros:

    average_of_reviews
0          166.679487


## Conclusiones

**Resumen de hallazgos:**

   - Tarea 1: Después del 1 de enero de 2000, se han publicado 819 libros según nuestra base de datos. 

   - Tarea 2: Los 20 libros con mayor cantidad de reseñas, tienen calificaciones que van desde 3.3 hasta 4.4. Al mismo tiempo, los libros con mejores calificaciones (de 5.0), suelen tener menos de 10 reseñas. 

   - Tarea 3: Las tres editoriales con mayor número de libros publicados son Penguin Books, Vintage y Grand Central Publishing. 

   - Tarea 4: Los cinco autores con las más altas calificaciones en promedio son `J.K. Rowling/Mary GrandPré`  `Agatha Christie` `Markus Zusak/Cao Xuân Việt Khương` `J.R.R. Tolkien` `Roald Dahl/Quentin Blake` 

   - Tarea 5:  El numero de reseñas que hace un usuario parece coincidir con el numero de calificaciones que realiza. En promedio, entre los usuarios que realizan más de 50 reseñas, los usuarios realizando 166.6 reseñas.

**Propuesta de valor:**

Podemos decir que sí vale la pena desarrollar aplicaciones para los amantes de la lectura. Los usuarios son activos a la hora de proporcionar calificaciones y reseñas. Podemos sugerir contactar las editoriales Penguin Books, Vintage y Grand Central Publishing por nuevos lanzamientos, y también ofrecer los clásicos de los autores `J.K. Rowling/Mary GrandPré`  `Agatha Christie` `Markus Zusak/Cao Xuân Việt Khương` `J.R.R. Tolkien` `Roald Dahl/Quentin Blake.`
